# Obtaining Reddit data: accessing the Reddit API
The first preliminary step to performing any kind of sentiment analysis on Reddit data is establishing a **Reddit instance** via **API**. This can be done with the **Python Reddit API Wrapper (PRAW)**, see documentation: https://praw.readthedocs.io/en/latest/

In order to access the API, a **Reddit account** is needed, login details below.
> E-mail: barbara.balcon@studbocconi.it
Username: thesis_3078976
Password: class_of_2021

I created an **application** at the following link: https://www.reddit.com/prefs/apps
> - Name: Sentiment Analysis
I selected 'script' (Script for personal use. Will only have access to the developers accounts
description)
- Description: Data will be used to perform sentiment analysis
- About url: blank
- Redirect url: http://www.example.com/unused/redirect/url (this is empty)

After creating the app, the following is generated.
> ID: L_wWPUNiMBmn1Q  
Secret: bUYKyV21W3xXK4GzXsY5tsNzpG3pRw



In [1]:
#import of the relevant libraries 
import praw
import pandas as pd
from praw.models import MoreComments

In [2]:
#create a reddit connection with reddit api details
reddit=praw.Reddit(client_id='L_wWPUNiMBmn1Q', client_secret='bUYKyV21W3xXK4GzXsY5tsNzpG3pRw', user_agent='ua')

The following pulls the **five hottest posts** in the subreddit wallstreetbets, printing the title and ID of each. This step is for illustrating purposes only and does *not* feed into the sentiment analysis performed later.

In [3]:
subreddit=reddit.subreddit('wallstreetbets')
for submission in subreddit.hot(limit=5):
    print(submission.title)
    print('Submission ID = ', submission.id, '\n')

Weekend Discussion Thread for the Weekend of April 01, 2021
Submission ID =  mi3t1v 

i'm about to YOLO my $800k life savings on starbucks gift cards, what are the tax implications ??
Submission ID =  mjy92g 

The phone numbers and personal data of over 553 million Facebook users have been published online for free
Submission ID =  mjllx5 

Institutions in GME increased 2% than Last month & RC owns 4 times more shares than George
Submission ID =  mjrc0t 

GME DD on April 2020 similarities to today
Submission ID =  mjf7mg 



# Storing the data: creating a SQL server and database

## Download, installation and setup of MySQL

I am about to collect data in a **SQL database**, I use **MySQL** as a database management system. In a nutshell I installed and set up MySQL on my machine (off-Python).

First, I **downloaded** the free version (Community Server, for macOS) of MySQL from the official website: https://dev.mysql.com/downloads/mysql/
I proceeded to **install** it using the wizard and to set up a password. I **started the server** in the MySQL panel in System preferences.

I also downloaded and installed **MySQL Shell**, again from the official website: https://dev.mysql.com/downloads/shell/

In order to use MySQL from the Terminal, I typed the following:  
`cd /usr/local/mysql/bin/; ls`  
and pressed Enter

Then:  
`./mysql -u root -p`    
pressed Enter and input the password when prompted to.

This turns the Terminal into a **MySQL Monitor: all SQL commands now work**. More of this in subsequent steps.

## Connecting to the server and creating a database
Getting back to the code, in the cell below we:
- import a package to deal with SQL;
- connect to the server;
- create a database: reddit_data;
- create a table within the database: reddit_data_sentiment.

A **server** can contain one or more **databases**, and each can store **tables** (just one of each in our case). The tables consist of **rows** and **columns**, columns are different characteristics of a post (author, body, etc.) and each row represents a post in r/wallstreetbets.

As of now, the server is hosted *locally* on my machine (hence host: localhost). The username (root) is default, while the password was set when installing MySQL.

In [9]:
import mysql.connector #importing the Python package to use MySQL
import datetime 
import time #packages for handling date and time data

#below I am connecting to the server I have started on my machine
mydb=mysql.connector.connect(host='localhost', 
                             user='root',
                            passwd='class_of_2021')

mycursor=mydb.cursor()

#creation of a database for storing reddit data 
mycursor.execute('CREATE DATABASE IF NOT EXISTS reddit_data')

#creating variables we are about to create instances of when analysing sentiment
#each is a column in the database
#they are all stored int he only tab le in the database, reddit_data_sentiment
mycursor.execute("""CREATE TABLE IF NOT EXISTS reddit_data.reddit_data_sentiment
                (date_time DATETIME,
                subreddit VARCHAR(500),
                title VARCHAR(500),        
                body VARCHAR(2000),
                author VARCHAR(500),
                sentiment DECIMAL(5,4)  
                )
                """)

#pushing the data to the database (we will use this variable in the next code snippet)
sqlFormula = "INSERT INTO reddit_data.reddit_data_sentiment (date_time, subreddit, title, body, author, sentiment) VALUES (%s, %s, %s, %s, %s, %s)"


# Performing post sentiment analysis and storing the results

## Introduction to VADER
Now we are ready to **live stream** comments from Reddit and perform sentiment analysis via **VADER** (Valence Aware Dictionary and sEntiment Reasoner). It is an open-source tool that was designed for social media specifically. It is lexicon and rule-based. 
See: 
> Hutto, C.J. & Gilbert, E.E. (2014). *VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text.* Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

It returns a **polarity score (-1, +1)** for each post: a normalized, weighted composite score that acts as a metric of the overall sentiment of a given post.  

## Storing the data in the database
In the *while* loop, we are **populating the table** in the SQL database we created earlier with data streamed from the Reddit API. For each post, we include:
- current date and time;
- subreddit (wallstreetbets);
- title;
- body;
- compound sentiment score.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer= SentimentIntensityAnalyzer() #just an abbreviation 
from unidecode import unidecode #a package for dealing with Unicode

while True:
    try:
        
        subreddit = reddit.subreddit('wallstreetbets')
        for comment in subreddit.stream.comments(skip_existing=True):
                current_time = datetime.datetime.now()
                subreddit = str(comment.subreddit)
                author = str(comment.author)
                title = str(comment.link_title)
                body = str(comment.body)
                if len(body) < 2000:
                    body = body
                elif len(body) > 2000:
                    body = "data is too large"
                
                vs = analyzer.polarity_scores(unidecode(body))
                sentiment = vs['compound'] #we are interested in the compound score
                db = (current_time,subreddit,title,body,author,sentiment)
                mycursor.execute(sqlFormula, db)
                mydb.commit()

    except Exception as e:
        print(str(e))
        time.sleep(10)
'''We keep an exception so that in case of error we do not hit
the API multiple times''' 

1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1
1406 (22001): Data too long for column 'body' at row 1


# Inspecting the results in the shell

We can use **MySQL Shell** (which we have already activated in the Terminal) to check that the data has indeed been stored. To do so, we go back to the Terminal and we input the following (note that SQl commands end with ;):

`show databases;`   
This will return all databases in the Terminal, including reddit_data.

`use reddit_data;`  
This will 'activate' the database, so that the following commands will be executed in reddit_data.

`select * from reddit_data_sentiment;`  
This returns all of the content of the reddit_data_sentiment table (note that * means 'everything' in SQL).

As of now, everything is running smoothly.